<a href="https://colab.research.google.com/github/kainlee2304/AdWaveOfFood/blob/master/ck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install pandas openpyxl

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
path =  "/content/drive/MyDrive/Colab Notebooks/"
os.chdir(path)
%pwd

Mounted at /content/drive


'/content/drive/MyDrive/Colab Notebooks'

In [5]:
import pandas as pd

# Đường dẫn tới file Excel đầu vào
excel_file = 'Dataset.xlsx'

# Đường dẫn tới file CSV đầu ra
csv_file = 'dataset.csv'

# Đọc file Excel
df = pd.read_excel(excel_file, engine='openpyxl')

# Xuất sang file CSV
df.to_csv(csv_file, index=False, encoding='utf-8')

print(f"Đã chuyển file {excel_file} sang {csv_file} thành công!")

Đã chuyển file Dataset.xlsx sang dataset.csv thành công!


In [14]:
df = pd.read_csv('dataset.csv')
df.head()

,Tên lớp học phần,Giảng viên,Thời khóa biểu,Phòng học,Tuần học,Sỉ số
0,An toàn ứng dụng Web và CSDL (1),ThS.Trần Thanh Liêm,Thứ Bảy | \t\t\t\t\tTiết 1->3,K.A107,"23->25,29->38",60.0
1,Automat và ngôn ngữ hình thức (1),ThS.Trần Đình Sơn,Thứ Sáu | \t\t\t\t\tTiết 8->9,K.A213,"23->25,28->39",68.0
2,Automat và ngôn ngữ hình thức (2),ThS.Dương Thị Mai Nga,Thứ Tư | \t\t\t\t\tTiết 3->4,K.A313,"24,25,28->39,41",68.0
3,Automat và ngôn ngữ hình thức (3),TS.Nguyễn Đức Hiển,Thứ Hai | \t\t\t\t\tTiết 1->2,K.A113,"23->25,29->40",68.0
4,Automat và ngôn ngữ hình thức (4),TS.Nguyễn Đức Hiển,Thứ Hai | \t\t\t\t\tTiết 3->4,K.A113,"23->25,29->40",68.0


In [19]:
import pandas as pd
import re

# Hàm tách cột "Tên lớp học phần" (đã sửa)
def split_course_name(course):
    course = str(course).strip()
    name, class_num, language, major, subtopic = '', '', 'Tiếng Việt', 'Không có', 'Không có'

    # Kiểm tra định dạng GDTC
    gdtc_match = re.match(r'GDTC\s*(\d+)\s*(?:\((.*?)\))?\s*(?:-(\d+))?', course)
    if gdtc_match:
        # Nếu là lớp GDTC
        major = gdtc_match.group(1)  # Số từ "GDTC [số]" (ví dụ: "1" từ "GDTC 1")
        name = f'GDTC {major}'  # Tên học phần: "GDTC 1"
        subtopic = gdtc_match.group(2) if gdtc_match.group(2) else 'Không có'  # Chủ đề phụ: "Điền kinh"
        class_num = gdtc_match.group(3) if gdtc_match.group(3) else ''  # Số lớp: "01"
        return pd.Series([name.strip(), class_num, language, major, subtopic])

    # Xử lý các trường hợp không phải GDTC (giữ nguyên logic cũ)
    class_match = re.search(r'\((\d+)\)', course)
    if class_match:
        class_num = class_match.group(1)
        name = course[:class_match.start()].strip()
    else:
        name = course

    if '_TA' in course:
        language = 'Tiếng Anh'
        course = course.replace('_TA', '').strip()
        name = name.replace('_TA', '').strip()

    parts = course.split('_')
    major_found = False
    for part in parts:
        if part in ['IT', 'CE', 'BA', 'JIT', 'KIT', 'GCE', 'GIC', 'NS', 'IR', 'DA', 'MC', 'SE', 'AD']:
            major = part
            major_found = True
        elif (part != class_num and part != 'TA' and part != name.split('(')[0].strip() and
              not re.match(r'\(\d+\)', part) and part not in [name, name + f' ({class_num})']):
            subtopic = part.strip()

    if '(' in name and ')' in name and not major_found:
        major_match = re.search(r'\((IT|CE|BA|JIT|KIT|GCE|GIC|NS|IR|DA|MC|SE|AD)\)', name)
        if major_match:
            major = major_match.group(1)
            name = name[:major_match.start()] + name[major_match.end():]

    if subtopic in [name, name + f' ({class_num})', course]:
        subtopic = 'Không có'

    return pd.Series([name.strip(), class_num, language, major, subtopic])

# Hàm tách cột "Thời khóa biểu"
def split_schedule(schedule):
    schedule = str(schedule).strip()
    day, periods = 'Không có', 'Không có'

    if '|' in schedule:
        day_part, period_part = schedule.split('|')
        day = day_part.strip()
        period_part = period_part.strip().replace('Tiết ', '')
        if '->' in period_part:
            start, end = map(int, period_part.split('->'))
            periods = ', '.join(str(i) for i in range(start, end + 1))
        else:
            periods = period_part
    return pd.Series([day, periods])

# Hàm tách cột "Phòng học"
def split_classroom(room):
    room = str(room).strip()
    area, room_num = 'Không có', 'Không có'

    if room == 'Chưa xếp phòng' or room == '(Chọn)':
        area = room
    elif '.' in room:
        area, room_num = room.split('.', 1)
    return pd.Series([area, room_num])

# Áp dụng các hàm tách cột
df[['Tên học phần', 'Lớp', 'Ngôn ngữ', 'Chuyên ngành', 'Chủ đề phụ']] = df['Tên lớp học phần'].apply(split_course_name)
df[['Thứ', 'Tiết']] = df['Thời khóa biểu'].apply(split_schedule)
df[['Khu vực', 'Số phòng']] = df['Phòng học'].apply(split_classroom)

# Kiểm tra các cột hiện có
print("Các cột trong DataFrame:", df.columns.tolist())

# Sửa tên cột 'Sỉ số' thành 'Sĩ số'
df_final = df[['Tên học phần', 'Lớp', 'Ngôn ngữ', 'Chuyên ngành', 'Chủ đề phụ',
               'Giảng viên', 'Thứ', 'Tiết', 'Khu vực', 'Số phòng', 'Tuần học', 'Sỉ số']]

# Xuất ra file Excel
output_file = 'newdataset.xlsx'
df_final.to_excel(output_file, index=False, engine='openpyxl')

print(f"Đã xuất dữ liệu ra file {output_file} thành công!")

# Nếu dùng Google Colab, thêm đoạn mã để tải file
try:
    from google.colab import files
    files.download('newdataset.xlsx')
except ImportError:
    print("Không phải môi trường Colab, kiểm tra file trong thư mục làm việc.")

# Hàm tách cột "Thời khóa biểu"
def split_schedule(schedule):
    schedule = str(schedule).strip()
    day, periods = 'Không có', 'Không có'

    if '|' in schedule:
        day_part, period_part = schedule.split('|')
        day = day_part.strip()
        period_part = period_part.strip().replace('Tiết ', '')
        if '->' in period_part:
            start, end = map(int, period_part.split('->'))
            periods = ', '.join(str(i) for i in range(start, end + 1))
        else:
            periods = period_part
    return pd.Series([day, periods])

# Hàm tách cột "Phòng học"
def split_classroom(room):
    room = str(room).strip()
    area, room_num = 'Không có', 'Không có'

    if room == 'Chưa xếp phòng' or room == '(Chọn)':
        area = room
    elif '.' in room:
        area, room_num = room.split('.', 1)
    return pd.Series([area, room_num])

# Áp dụng các hàm tách cột
df[['Tên học phần', 'Lớp', 'Ngôn ngữ', 'Chuyên ngành', 'Chủ đề phụ']] = df['Tên lớp học phần'].apply(split_course_name)
df[['Thứ', 'Tiết']] = df['Thời khóa biểu'].apply(split_schedule)
df[['Khu vực', 'Số phòng']] = df['Phòng học'].apply(split_classroom)

# Kiểm tra các cột hiện có
print("Các cột trong DataFrame:", df.columns.tolist())

# Giữ nguyên các cột khác
# Sửa tên cột 'Sĩ số' nếu cần (dựa trên output của df.columns)
df_final = df[['Tên học phần', 'Lớp', 'Ngôn ngữ', 'Chuyên ngành', 'Chủ đề phụ',
               'Giảng viên', 'Thứ', 'Tiết', 'Khu vực', 'Số phòng', 'Tuần học', 'Sỉ số']]

# Xuất ra file Excel
output_file = 'newdataset.xlsx'
df_final.to_excel(output_file, index=False, engine='openpyxl')

print(f"Đã xuất dữ liệu ra file {output_file} thành công!")

Các cột trong DataFrame: ['Tên lớp học phần', 'Giảng viên', 'Thời khóa biểu', 'Phòng học', 'Tuần học', 'Sỉ số', 'Tên học phần', 'Lớp', 'Ngôn ngữ', 'Chuyên ngành', 'Chủ đề phụ', 'Thứ', 'Tiết', 'Khu vực', 'Số phòng']
Đã xuất dữ liệu ra file newdataset.xlsx thành công!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Các cột trong DataFrame: ['Tên lớp học phần', 'Giảng viên', 'Thời khóa biểu', 'Phòng học', 'Tuần học', 'Sỉ số', 'Tên học phần', 'Lớp', 'Ngôn ngữ', 'Chuyên ngành', 'Chủ đề phụ', 'Thứ', 'Tiết', 'Khu vực', 'Số phòng']
Đã xuất dữ liệu ra file newdataset.xlsx thành công!
